python装饰器，表示为内包函数中使用新定义的函数代替参数函数，通过@表示装饰函数，@下面为替代函数

In [12]:
def add(a : int, b : int):
    return a + b

def multiply(a : int, b : int):
    return a * b

def minus(a : int, b : int):
    return a - b

In [13]:
def printFuncName(func):
    def decorator(*args, **kwargs):
        print(f"\033[33mcall {func.__name__}\033[0m")
        return func(*args, **kwargs)
    return decorator

In [14]:
@printFuncName
def add(a : int, b : int):
    return a + b

@printFuncName
def multiply(a : int, b : int):
    return a * b

@printFuncName
def minus(a : int, b : int):
    return a - b

print(add(1, 2))
print(multiply(1, 2))
print(minus(1, 2))

call add
3
call multiply
2
call minus
-1


如果你的项目中有许多可能会随处用到的函数和类，你希望把所有的这些指定的函数和类整合到一个字典（字典的key是函数名，类名或者自定义的名字，value是对应的函数对象或者类）中，那么这样一个过程称为注册。如果我将一个函数放入了这个字典中，那么就称完成了对该函数的注册。

In [15]:
register = {}

def func1():
    pass

f = lambda x : x

class cls1(object):
    pass

register[func1.__name__] = func1
register[f.__name__] = f
register[cls1.__name__] = cls1

但是这样做非常的臃肿，添加新的函数的时候也不方便，需要一种自动注册的方法，就需要用到装饰器，写一个类Register，这个类的register方法充当我们的要实现注册功能的装饰器函数，由于注册的载体往往是字典，因此，我们这个Register类继承dict。

In [28]:
class Register(dict):
    def __init__(self, *args, **kwargs):
        super(Register, self).__init__(*args, **kwargs)
        self._dict = {}
    
    def register(self, target):
        def add_register_item(key, value):
            if not callable(value):
                raise Exception(f"register object must be callable! But receice:{value} is not callable!")
            if key in self._dict:
                print(f"warning: \033[33m{value.__name__} has been registered before, so we will overriden it\033[0m")
            self[key] = value
            return value

        if callable(target):            # 如果传入的目标可调用，说明之前没有给出注册名字，我们就以传入的函数或者类的名字作为注册名
            return add_register_item(target.__name__, target)
        else:                           # 如果不可调用，说明额外说明了注册的可调用对象的名字
            return lambda x : add_register_item(target, x)
    
    def __setitem__(self, key, value):
        self._dict[key] = value

    def __getitem__(self, key):
        return self._dict[key]
    
    def __contains__(self, key):
        return key in self._dict
    
    def __str__(self):
        return str(self._dict)
    
    def keys(self):
        return self._dict.keys()
    
    def values(self):
        return self._dict.values()
    
    def items(self):
        return self._dict.items()
    
    def __call__(self, target):
        return self.register(target)
        

需要实现功能：如果用户不填入参数，那么就以被注册函数的函数名作为注册名，若填入参数，则以该参数作为该函数的注册名。

我就可以将实例后的对象的register方法作为装饰器函数套在我们想要注册的函数或者类上面啦！

In [29]:
register_functions = Register()

@register_functions.register
def add(a : int, b : int):
    return a + b

@register_functions.register("my multiply")
def multiply(a : int, b : int):
    return a * b

@register_functions.register
def minus(a : int, b : int):
    return a - b

In [30]:
if __name__ == "__main__":
    for k, v in register_functions.items():
        print(f"key: {k}, value: {v}")

key: add, value: <function add at 0x00000260395DEF70>
key: my multiply, value: <function multiply at 0x000002603B16E9D0>
key: minus, value: <function minus at 0x000002603B16E820>


In [31]:
print(register_functions)

{'add': <function add at 0x00000260395DEF70>, 'my multiply': <function multiply at 0x000002603B16E9D0>, 'minus': <function minus at 0x000002603B16E820>}
